In this notebook I will try to beat our accuracy score of 76% from the previous notebook.

In [1]:
import random
import numpy as np
import matplotlib as plt
import tensorflow as tf

# Data Importing and Preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !unzip gdrive/MyDrive/archive.zip

In [4]:
import os

num_cat_images_train = len(os.listdir("training_set/training_set/cats"))
num_dog_images_train = len(os.listdir("training_set/training_set/dogs"))
num_cat_images_test = len(os.listdir("test_set/test_set/cats"))
num_dog_images_test = len(os.listdir("test_set/test_set/dogs"))

In [5]:
print(f"Training dataset has {num_cat_images_train} cat images and {num_dog_images_train} dog images.")
print(f"Test dataset has {num_cat_images_test} cat images and {num_dog_images_test} dog images.")

Training dataset has 4001 cat images and 4006 dog images.
Test dataset has 1012 cat images and 1013 dog images.


In [6]:
# Get the class names (programmatically, this is much more helpful with a longer list of classes)

class_names = [ item for item in os.listdir('test_set/test_set') if os.path.isdir(os.path.join("test_set/test_set", item)) ]
print(class_names)

['cats', 'dogs']


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Normalization
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "training_set/training_set/"
test_dir = "test_set/test_set/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

test_data = valid_datagen.flow_from_directory(test_dir,
                                              batch_size=32,
                                              target_size=(224, 224),
                                              class_mode="binary",
                                              seed=42)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [8]:
"""
    This will flip randomly rotate our images between 0 and 120 degrees, zoom
    them a little bit, shift them across their width and height and will also
    horizontally flip them randomly.
"""

train_datagen_augmented = ImageDataGenerator(rescale = 1/255,
                                          rotation_range=20,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          horizontal_flip=True)

# Create ImageDataGenerator training instance without data augmentation
train_datagen = ImageDataGenerator(rescale=1/255.) 

# Create ImageDataGenerator test instance without data augmentation
test_datagen = ImageDataGenerator(rescale=1/255.)

In [9]:
# Import data and augment it from training directory
print("Augmented training images:")
train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                   target_size=(224, 224),
                                                                   batch_size=32,
                                                                   class_mode='binary',
                                                                   shuffle=False)

# Create non-augmented data batches
print("Non-augmented training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='binary',
                                               shuffle=False) # Don't shuffle for demonstration purposes

print("Unchanged test images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='binary')

Augmented training images:
Found 8005 images belonging to 2 classes.
Non-augmented training images:
Found 8005 images belonging to 2 classes.
Unchanged test images:
Found 2023 images belonging to 2 classes.


In [10]:
# Get data batch samples
images, labels = train_data.next()
augmented_images, augmented_labels = train_data_augmented.next()

In [11]:
# Import data and augment it from directories
train_data_augmented_shuffled = train_datagen_augmented.flow_from_directory(train_dir,
                                                                            target_size=(224, 224),
                                                                            batch_size=32,
                                                                            class_mode='binary',
                                                                            shuffle=True) # Shuffle data (default)

Found 8005 images belonging to 2 classes.


# Model 1 : CNN

In [12]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

# Create the model
model_1 = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3), kernel_initializer='he_uniform'),
  MaxPool2D(pool_size=2),
  Conv2D(64, 3, activation='relu', kernel_initializer='he_uniform'),
  MaxPool2D(),
  Conv2D(128, 3, activation='relu', kernel_initializer='he_uniform'),
  MaxPool2D(),
  Flatten(),
  Dense(128, activation="relu", kernel_initializer='he_uniform'),
  Dense(1, activation='sigmoid')
])

# Compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
                metrics=['accuracy'])

# Fit the model
history_1 = model_1.fit(train_data_augmented_shuffled,
                        epochs=10,
                        steps_per_epoch=len(train_data_augmented),
                        validation_data=test_data,
                        validation_steps=0.5*len(test_data))

Epoch 1/10
251/251 [==============================] - 118s 419ms/step - loss: 0.7022 - accuracy: 0.5462 - val_loss: 0.6706 - val_accuracy: 0.5947
Epoch 2/10
251/251 [==============================] - 104s 413ms/step - loss: 0.6691 - accuracy: 0.5800 - val_loss: 0.6849 - val_accuracy: 0.5469
Epoch 3/10
251/251 [==============================] - 105s 418ms/step - loss: 0.6514 - accuracy: 0.6140 - val_loss: 0.6679 - val_accuracy: 0.5449
Epoch 4/10
251/251 [==============================] - 102s 407ms/step - loss: 0.6368 - accuracy: 0.6320 - val_loss: 0.6911 - val_accuracy: 0.5605
Epoch 5/10
251/251 [==============================] - 103s 412ms/step - loss: 0.6225 - accuracy: 0.6472 - val_loss: 0.5809 - val_accuracy: 0.6953
Epoch 6/10
251/251 [==============================] - 102s 406ms/step - loss: 0.6111 - accuracy: 0.6622 - val_loss: 0.5703 - val_accuracy: 0.6992
Epoch 7/10
251/251 [==============================] - 101s 404ms/step - loss: 0.6053 - accuracy: 0.6668 - val_loss: 0.5557 -

# Model 2 : 50 Epochs

In [21]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import keras
from matplotlib import pyplot
# from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generators
	train_datagen = ImageDataGenerator(rescale=1.0/255.0,
		width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	test_datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = train_datagen.flow_from_directory('training_set/training_set/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = test_datagen.flow_from_directory('test_set/test_set/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it),
                     validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1)
	# evaluate model
	_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Epoch 1/50
126/126 [==============================] - 88s 697ms/step - loss: 0.7095 - accuracy: 0.5344 - val_loss: 0.6823 - val_accuracy: 0.5709
Epoch 2/50
126/126 [==============================] - 87s 688ms/step - loss: 0.6809 - accuracy: 0.5639 - val_loss: 0.6703 - val_accuracy: 0.6011
Epoch 3/50
126/126 [==============================] - 87s 694ms/step - loss: 0.6682 - accuracy: 0.5875 - val_loss: 0.7172 - val_accuracy: 0.5166
Epoch 4/50
126/126 [==============================] - 86s 682ms/step - loss: 0.6619 - accuracy: 0.5913 - val_loss: 0.6392 - val_accuracy: 0.6352
Epoch 5/50
126/126 [==============================] - 86s 683ms/step - loss: 0.6484 - accuracy: 0.6144 - val_loss: 0.6360 - val_accuracy: 0.6357
Epoch 6/50
126/126 [==============================] - 87s 693ms/step - loss: 0.6338 - accuracy: 0.6411 - val_loss: 0.6415 - val_accuracy: 0.6303
Epoch 7/50
126/126 [==========================

So training for 50 epochs did improve our model performance. Now lets try transfer learning.

# Model 3 : Transfer Learning

In [30]:
import tensorflow as tf

In [37]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
# from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
	# load model
	model_1 = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model_1.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model_1.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model_1 = Model(inputs=model_1.inputs, outputs=output)
	# compile model
	opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
	model_1.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model_1

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model_1 = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('training_set/training_set/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	test_it = datagen.flow_from_directory('test_set/test_set/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	history_1 = model_1.fit(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
	# evaluate model
	_, acc = model_1.evaluate(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history_1)

# entry point, run the test harness
run_test_harness()

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Epoch 1/10
126/126 [==============================] - 65s 474ms/step - loss: 0.5506 - accuracy: 0.9490 - val_loss: 0.0831 - val_accuracy: 0.9698
Epoch 2/10
126/126 [==============================] - 53s 421ms/step - loss: 0.0257 - accuracy: 0.9909 - val_loss: 0.0867 - val_accuracy: 0.9713
Epoch 3/10
126/126 [==============================] - 53s 416ms/step - loss: 0.0063 - accuracy: 0.9985 - val_loss: 0.1010 - val_accuracy: 0.9723
Epoch 4/10
126/126 [==============================] - 53s 421ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.1120 - val_accuracy: 0.9718
Epoch 5/10
126/126 [==============================] - 52s 415ms/step - loss: 0.0012 - accuracy: 0.9999 - val_loss: 0.1226 - val_accuracy: 0.9713
Epoch 6/10
126/126 [==============================] - 52s 415ms/step - loss: 8.5822e-04 - accuracy: 0.9998 - val_loss: 0.1279 - val_accuracy: 0.9723
Epoch 7/10
126/126 [======================

Wooohoooooo 97% accuracy.